# Normal Equations

### Constructing Normal Equations 

Consider the fixed linear model

$$
\mathbf{y} = \mathbf{X\beta} + \mathbf{e},
$$

where $\mathbf{y}$ is a $n\times1$ vector of observations, $\mathbf{X}$ is a $n\times p$ known matrix, $\mathbf{\beta}$ is a $p\times 1$ vector of fixed effects, and $\mathbf{e}$ is an $n\times 1$ vector of residuals that are often assumed to be identically and independently distributed with null mean and variance $\sigma^2_e$. 

The normal equations for this model are

$$
(\mathbf{X}'\mathbf{X})\hat{\mathbf{\beta}} = \mathbf{X}'\mathbf{y},
$$

where  $\mathbf{X'X}$ is $p\times p$ and  $\mathbf{X'y}$ is $p\times 1$. In breeding applications, $p$ may be large, but  $\mathbf{X'X}$ is often very sparse, i.e., has very few non-zero elements. Efficient algorithms take advantage of this sparse structure to reduce computing time and storage requirements.

In this section, we will consider how to compute a **full-stored**  $\mathbf{X'X}$ matrix and $\mathbf{X'y}$ vector.

* One way to proceed would be to first build $\mathbf{X}$ and then get $\mathbf{X'X}$ by matrix multiplication. 
* Next, we will look more closely at a one-way model and write a simple program for constructing the normal equations for this model.

#### Data

Consider the following data from a hypothetical one-way experiment with four levels of one factor.

In [2]:
using DataFrames
data = DataFrame(x=[1,1,2,2,2,2,3,3,4,1],y=[1.1,1.2,1.9,1.2,2.0,1.7,1.0,1.7,1.1,1.7])

,x,y
,Int64,Float64
1,1,1.1
2,1,1.2
3,2,1.9
4,2,1.2
5,2,2.0
6,2,1.7
7,3,1.0
8,3,1.7
9,4,1.1


### One way model

The $\mathbf{X}$ matrix for the one-way model

$$
y_{ij} = \mu + \alpha_i + e_{ij}
$$

is

$$
\mathbf{X} = 
\begin{bmatrix}
1 & 1 & 0 & 0 & 0 \\
1 & 1 & 0 & 0 & 0 \\
1 & 0 & 1 & 0 & 0 \\
1 & 0 & 1 & 0 & 0 \\
1 & 0 & 1 & 0 & 0 \\
1 & 0 & 1 & 0 & 0 \\
1 & 0 & 0 & 1 & 0 \\
1 & 0 & 0 & 1 & 0 \\
1 & 0 & 0 & 0 & 1 \\
1 & 1 & 0 & 0 & 0 \\
\end{bmatrix}
$$

Note $\mathbf{X'X}$ is singular, but a solution can be obtained as follows. 

### Computing X'X as the product of full-stored X' and X

In [3]:
n = size(data,1)
p = length(unique(data[:x]))
X = zeros(n,p);

In [4]:
@time for i = 1:n
    j = data[:x][i]
    X[i,j] = 1.0
end

  0.003962 seconds (36 allocations: 1.984 KiB)


In [5]:
X = [ones(n) X]

10×5 Array{Float64,2}:
 1.0  1.0  0.0  0.0  0.0
 1.0  1.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  1.0
 1.0  1.0  0.0  0.0  0.0

In [6]:
y = data[:y];

In [7]:
lhs = X'X
rhs = X'y;

In [8]:
sol = lhs\rhs

5-element Array{Float64,1}:
 -0.04583333333333304
  1.3791666666666667 
  1.745833333333333  
  1.3958333333333333 
  1.1458333333333333 

#### Verify solution 

In [9]:
[lhs*sol rhs]

5×2 Array{Float64,2}:
 14.6  14.6
  4.0   4.0
  6.8   6.8
  2.7   2.7
  1.1   1.1

In this section, we will consider how to compute a **sparse-stored**  $\mathbf{X'X}$ matrix and $\mathbf{X'y}$ vector. 

* One way to proceed would be to first build $\mathbf{X}$ as a sparse matrix and then get $\mathbf{X'X}$ by matrix multiplication. 
* Now we will use the same one-way model and write a simple program for constructing the normal equations for this model.

### Computing X'X as the product of sparse-stored X' and X

In [17]:
using SparseArrays, LinearAlgebra
X = sparse(1:n,data[:x],1.0);

In [18]:
X =[ones(n) X]

10×5 SparseMatrixCSC{Float64,Int64} with 20 stored entries:
  [1 ,  1]  =  1.0
  [2 ,  1]  =  1.0
  [3 ,  1]  =  1.0
  [4 ,  1]  =  1.0
  [5 ,  1]  =  1.0
  [6 ,  1]  =  1.0
  [7 ,  1]  =  1.0
  [8 ,  1]  =  1.0
  [9 ,  1]  =  1.0
  [10,  1]  =  1.0
  [1 ,  2]  =  1.0
  [2 ,  2]  =  1.0
  [10,  2]  =  1.0
  [3 ,  3]  =  1.0
  [4 ,  3]  =  1.0
  [5 ,  3]  =  1.0
  [6 ,  3]  =  1.0
  [7 ,  4]  =  1.0
  [8 ,  4]  =  1.0
  [9 ,  5]  =  1.0

In [19]:
Matrix(X)

10×5 Array{Float64,2}:
 1.0  1.0  0.0  0.0  0.0
 1.0  1.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  1.0
 1.0  1.0  0.0  0.0  0.0

In [13]:
lhs = X'X
rhs = X'y

5-element Array{Float64,1}:
 14.599999999999998
  4.0              
  6.8              
  2.7              
  1.1              

In [25]:
QRLhs = qr(lhs) 
sol1 = QRLhs\rhs

5-element Array{Float64,1}:
 1.099999999999999  
 0.23333333333333395
 0.6000000000000002 
 0.25000000000000117
 0.0                

In [20]:
[lhs*sol1 rhs]

5×2 Array{Float64,2}:
 14.6  14.6
  4.0   4.0
  6.8   6.8
  2.7   2.7
  1.1   1.1

In [21]:
spRhs = sparse(rhs)

5-element SparseVector{Float64,Int64} with 5 stored entries:
  [1]  =  14.6
  [2]  =  4.0
  [3]  =  6.8
  [4]  =  2.7
  [5]  =  1.1

In [31]:
k = [0.0;1.0;-1.0;0;0.0]
b = QRLhs\k
round.(lhs*b,digits=3)

5-element Array{Float64,1}:
 -0.0
  1.0
 -1.0
 -0.0
 -0.0

### [Supplemental Note: Computing full-stored or sparse-stored X'X without matrix multiplication](3.2.SupplementalNote.ipynb)